Import the packages including jax, equinox and diffrax

In [4]:
from typing import Union
import pdb
import diffrax
import equinox as eqx  # https://github.com/patrick-kidger/equinox
import jax
import jax.nn as jnn
import jax.numpy as jnp
import jax.random as jr
import jax.tree_util as jtu
import matplotlib.pyplot as plt
import optax  # https://github.com/deepmind/optax
import numpy as np
import control
import pdb
import jax
from jax import lax
import lineax
from helpers_impl import generate_system,  NeuralODEPlant, get_gpu_memory
from tqdm import tqdm
import gc
from IPython.display import clear_output
from jax.flatten_util import ravel_pytree
from diffrax import diffeqsolve, Tsit5, ODETerm, ReversibleHeun, Heun
#Set 64 precision
jax.config.update("jax_enable_x64", True)

The block of code for initialization of Neural ODE and optimizer

In [6]:
#number of dimensions 
data_size=2


#initialize the dynamical system
A=np.array([[1,0],[1,0]])
B=np.array([[1],[0]])
x00=np.array([1,0.5])
%store  A
%store   B
%store  x00
print(A)
print(B)
print(x00)
batch_size=1  

t_size=32
t0 = 0
t1 = 1
ts = jnp.linspace(t0, t1, t_size)
tss = jnp.expand_dims(ts, axis=0)
tsss = jnp.repeat(tss, repeats=batch_size, axis=0)
#initialize the ODE
seed=5679

hidden_size=data_size
noise_size=hidden_size+1
width_size=10
depth=5

key = jr.PRNGKey(seed)

generator = NeuralODEPlant(
    data_size,
    hidden_size,
    width_size,
    depth,
    A,
    B,
    key=key,
)


#learning rate
learning_rate=0.00001

#adam optimizer
b1=0.9
b2=0.999
eps=1e-08
optim = optax.adam(learning_rate, b1=b1, b2=b2, eps=0, eps_root=eps)

opt_state = optim.init(eqx.filter(generator, eqx.is_inexact_array))
#control energy penalty
R=0.0


#initial conditions 
y00=jnp.append(x00, jnp.zeros((1,1)))
y0 = jnp.expand_dims(y00, axis=0)

Stored 'A' (ndarray)
Stored 'B' (ndarray)
Stored 'x00' (ndarray)
[[1 0]
 [1 0]]
[[1]
 [0]]
[1.  0.5]


Functions for evaluation of gradients of loss and control energy

In [ ]:
#common denominator term
@jax.jit
def smoothed_inverse_grad(grad, eps=1e-05):
    return 1.0 / jnp.sqrt(grad + eps)

#for cost and its gradient evaluation
@eqx.filter_value_and_grad  
def loss(model):
   
    pred_y = jax.vmap(model)(tsss,  y0)
    
    los_energy = jnp.mean(jnp.sum(pred_y[:,-1,0:hidden_size]**2,axis=1))

    return los_energy

#for energy and its gradient evaluation
@eqx.filter_value_and_grad  
def loss_kin(model):
   
    pred_y = jax.vmap(model)(tsss,  y0)
    
    kin_energy=  jnp.mean(pred_y[:,-1,-1])

    return kin_energy

#for Hessian evaluation
@eqx.filter_jit
def l1(flat_params, static, unravel_fn):

    unflat_params = unravel_fn(flat_params)
    modelx = eqx.combine(static, unflat_params)

    vf =  diffrax.ODETerm(modelx.vf)

    solver = Heun(scan_kind="bounded")
    t0 = ts[0]
    t1 = ts[-1]
   
    dt=(t1-t0)/ts.shape[0]
    pdb.set_trace()
    pred_y = diffeqsolve(vf, solver, t0=t0, t1=t1, dt0=dt, y0=y00, args=None)
    
    los_energy = jnp.mean(jnp.sum(pred_y.ys[-1,0:hidden_size]**2,axis=0))
    score= los_energy

    return score

#for evaluation of directional derivatives of control energy
@jax.jit
def rms_full_grad(flat_grads, flat_grads1, hessian,  params, eps, ema_grad, ema_sqr_grad, counter):
    #common denominator term
    ema_sqr_grad = ema_sqr_grad /(1-b2**(counter+1))
    ema_grad = ema_grad /(1-b1**(counter+1))
    denom= smoothed_inverse_grad((ema_sqr_grad), eps)
    dem2 = smoothed_inverse_grad((jnp.square(flat_grads)), eps)
    

    PP_ema= ema_grad * (hessian @ (ema_grad* denom) )

    P_multiplier=  learning_rate* denom * ((1+b1)/(1-b1)  +((1+b2)/(1-b2))* ( eps*denom**2 -1 ))/2 


    kin_decline_from_loss = -jnp.dot(flat_grads1, (ema_grad* denom)) 
    kin_decline_from_P_term = - jnp.dot(flat_grads1, PP_ema * P_multiplier ) 
 

    jax.debug.print("directional derivative along unmodified loss = {}", kin_decline_from_loss)
    jax.debug.print("directional derivative along bias term = {}",  kin_decline_from_P_term)
 
    return kin_decline_from_loss, kin_decline_from_P_term
    
@eqx.filter_jit
def make_step(model, batch_size, tsss, key, opt_state, y0,  ema_grad, ema_sqr_grad, counter):

    key = jr.split(key, batch_size)
    
    #evaluate loss, energy and gradients
    value, grads = loss(model)
    value1, grads1 = loss_kin(model)
    #hessian evaluation
    arr, static = eqx.partition(model, eqx.is_inexact_array)
    params, unravel_fn = ravel_pytree(arr)
    hessian=jax.hessian(l1)(params, static, unravel_fn)
 
    #flattening the gradients
    flat_grads, _ = ravel_pytree(eqx.filter(grads, eqx.is_inexact_array))
    flat_grads1, _ = ravel_pytree(eqx.filter(grads1, eqx.is_inexact_array))

    #evaluation of ema gradients
    ema_grad = b1*ema_grad + (1-b1) * flat_grads
    ema_sqr_grad = b2*ema_sqr_grad + (1-b2) * jnp.square(flat_grads)

    cos_theta_grad_loss, cos_theta_grad_kin=rms_full_grad(flat_grads, flat_grads1, hessian, params, eps, ema_grad, ema_sqr_grad, counter)

    #update the control network
    updates, opt_state = optim.update(grads, opt_state)
    model = eqx.apply_updates(model, updates)

    counter+=1
    return model, opt_state, grads, value, value1, cos_theta_grad_kin, cos_theta_grad_loss, jnp.linalg.norm(params), ema_grad, ema_sqr_grad, counter

The loop for evaluation of control policy, loss and control energy, directional derivatives 

In [7]:
#compute the first values of cost and energy for ema 
tot_iter=100000
losses, grads = loss(generator)
kin, grads1 = loss_kin(generator)
arr, static = eqx.partition(generator, eqx.is_inexact_array)
params, unravel_fn = ravel_pytree(arr)
flat_grads, _ = ravel_pytree(eqx.filter(grads, eqx.is_inexact_array))
ema_grad =  flat_grads
ema_sqr_grad = jnp.square(flat_grads)


#save costs and energy
saved_loss=[]
saved_kin=[]
#save directional derivatives
saved_product1=[]
saved_product2=[]

ini_key=jr.PRNGKey(seed)
key= jr.split(key, 1)[0]
alpha=0.9

counter=jnp.int32(0)
tsss = jnp.repeat(tss, repeats=batch_size, axis=0)

score=10000000
epochs=[]
ema_loss=0
for k in tqdm(range(0,tot_iter)):
   
    key= jr.split(key, 1)[0]
    #run optimization steps, evaluate the cost and energy, directional derivatives
    generator, opt_state, grads, losses, kin, cos_theta_imp, cos_theta_loss, param_norm, ema_grad, ema_sqr_grad, counter= make_step(generator, batch_size, tsss, key, opt_state, y0,  ema_grad, ema_sqr_grad,counter)
    
    if k==0:
        ema_loss=losses
    else:
        ema_loss=ema_loss*alpha+losses*(1-alpha)
    
    if ema_loss<score:
        score=ema_loss

    print("ema loss")
    print(ema_loss)
    print("kin")
    print(kin)
    saved_loss.append(jax.device_get(losses))
    saved_kin.append(jax.device_get(kin))
    saved_product1.append(jax.device_get(cos_theta_imp))
    saved_product2.append(jax.device_get(cos_theta_loss))

    if k%500==0:
        gc.collect()
        clear_output(wait=True)
        
    if jnp.linalg.norm(ema_loss)<1e-06:
        break
    del grads, losses, kin



  0%|          | 0/100000 [00:00<?, ?it/s]

> /tmp/ipykernel_1820067/3142518197.py(99)l1()
     97     dt=(t1-t0)/ts.shape[0]
     98     pdb.set_trace()
---> 99     pred_y = diffeqsolve(vf, solver, t0=t0, t1=t1, dt0=dt, y0=y00, args=None)
    100 
    101     los_energy = jnp.mean(jnp.sum(pred_y.ys[-1,0:hidden_size]**2,axis=0))



ipdb>  vf


ODETerm(
  vector_field=VectorField(
    poly0=MLP(
      layers=(
        Linear(
          weight=f32[10,1],
          bias=f32[10],
          in_features=1,
          out_features=10,
          use_bias=True
        ),
        Linear(
          weight=f32[10,10],
          bias=f32[10],
          in_features=10,
          out_features=10,
          use_bias=True
        ),
        Linear(
          weight=f32[10,10],
          bias=f32[10],
          in_features=10,
          out_features=10,
          use_bias=True
        ),
        Linear(
          weight=f32[10,10],
          bias=f32[10],
          in_features=10,
          out_features=10,
          use_bias=True
        ),
        Linear(
          weight=f32[10,10],
          bias=f32[10],
          in_features=10,
          out_features=10,
          use_bias=True
        ),
        Linear(
          weight=f32[1,10],
          bias=f32[1],
          in_features=10,
          out_features=1,
          use_bias=True
       

ipdb>  y00


Array([1. , 0.5, 0. ], dtype=float32)


ipdb>  exit


  0%|          | 0/100000 [11:38<?, ?it/s]
